In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 400)
pd.set_option("display.max_rows", 700)

In [ ]:
import editdistance

In [ ]:
from collections import Counter

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
import json
from twython import Twython
from twython import TwythonError, TwythonRateLimitError

# 1. Load data

## a.) Load mention events

In [ ]:
mentions_df = pd.read_csv("/mnt/idms/fberes/network/roland_garros/data/rg17_mentions_with_names.csv", sep="|")

In [ ]:
mentions_df.head()

In [ ]:
print(len(mentions_df))
trg_mentions_df = mentions_df.drop_duplicates(subset=["trg_str", "trg_screen_str"])
print(len(trg_mentions_df))
print(trg_mentions_df["trg_str"].nunique())
print(trg_mentions_df["trg_screen_str"].nunique())

In [ ]:
trg_mentions_df["trg_str"].value_counts().head(10)

In [ ]:
trg_mentions_df[trg_mentions_df["trg_str"] == "Matt"]

In [ ]:
trg_dict = {}
for ix in trg_mentions_df.index:
    trg_str = trg_mentions_df.ix[ix]["trg_str"]
    if trg_str not in trg_dict:
        trg_dict[trg_str] = []
    trg_dict[trg_str].append(trg_mentions_df.ix[ix]["trg_screen_str"])
    
print(len(trg_dict))

In [ ]:
trg_dict_values = sum(trg_dict.values(), [])
trg_dict_values = list(filter(lambda value: pd.notnull(value), trg_dict_values))

In [ ]:
with open("../trg_dict_values.json", "w") as f:
    json.dump(trg_dict_values, f)

In [ ]:
screen_dict_values = dict(zip(trg_mentions_df["trg_screen_str"], trg_mentions_df["trg_str"]))

def check_tennis(screen_name):
    try:
        return ("tennis" in tw.show_user(screen_name=screen_name)["description"].lower())
    except TwythonRateLimitError:
        raise
    except TwythonError:
        
        non_existing_users.append(screen_name)

trg_dict

## b.) Load schedule events

In [ ]:
schedule_df = pd.read_csv("/mnt/idms/fberes/network/roland_garros/data/schedule_df.csv", sep="|")

In [ ]:
schedule_df.head()

# 2. Find tennis player matches

## a.) Collect players from mention and schedule events

In [ ]:
c_men = mentions_df["trg_str"].value_counts()
c_men_names = c_men.index

In [ ]:
active_list = list(schedule_df["playerName active"])
opponent_list = list(schedule_df["playerName opponent"])

In [ ]:
c_sch = Counter(active_list+opponent_list)

c_sch.most_common()

## b.) Find best matches with edit distance

In [ ]:
len(c_men), len(c_sch)

In [ ]:
%%time
row_size = len(c_sch)
edit_distances = np.zeros( (row_size, len(c_men)) )
player_names = list(c_sch.keys())
for i in range(row_size):
    reference = player_names[i]
    #print(reference)
    edit_distances[i,:] = [editdistance.eval(reference.lower(), comp.lower()) for comp in c_men_names]

In [ ]:
min_indices = edit_distances.argmin(axis=1)
player_matches = []
for i in range(row_size):
    reference, min_match = player_names[i], c_men_names[min_indices[i]]
    edit_dist = editdistance.eval(reference.lower(), min_match.lower())
    player_matches.append((reference, c_sch[reference], min_match, edit_dist))
player_matches_df = pd.DataFrame(player_matches, columns=["schedule_name", "schedule_count", "twitter_name", "distance"])

In [ ]:
player_matches_df.insert(3, "trg_screen_str", player_matches_df["twitter_name"].apply(lambda key: trg_dict[key])) 
player_matches_df.head()

### Edit distance distribution for best matches

In [ ]:
player_matches_df["distance"].value_counts()

### Collect true matches

**Notes on name matches:** (approx: 317+29+13 matches = **359 out of 628**)
   * names with distance 1 are (pure matches)
   * with distance 2 some name must be excluded (Di Wu -?! Kai Wu)
   * with distance 3 there are name matches (Matthew Ebden - Matt Ebden) but most of them are a dismatch
   * in distance 4 I found only a few match: (Tomas Machac - Tomasz Maćczak),(Harmony Tan - Harmony)
   * in distance 5 could be a match: (Zarina Diyas - Team Zarina Diyas) DE pl. vkiket kézzel kell megtalálni: (Leonardo Mayer NEM Leo Mayer HANEM @tennismayer - viszont ez a user 1000 éve inaktív..)

In [ ]:
true_matches = {}

#### i.) Pure matches

In [ ]:
zero_dist_matches = player_matches_df[player_matches_df["distance"] == 0]

In [ ]:
update_dict = dict(zip(zero_dist_matches["schedule_name"], zero_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### ii.) 1 distance matches

In [ ]:
one_dist_matches = player_matches_df[player_matches_df["distance"]==1]

In [ ]:
update_dict = dict(zip(one_dist_matches["schedule_name"], one_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### iii.) 2 distance matches

In [ ]:
two_dist_matches = player_matches_df[player_matches_df["distance"] == 2]

In [ ]:
two_dist_matches

In [ ]:
name_list = ["Sofia Kenin", "Michael Mmoh", "Francesco Forti", "Di Wu"]
two_dist_matches = two_dist_matches[~two_dist_matches["schedule_name"].isin(name_list)]

In [ ]:
update_dict = dict(zip(two_dist_matches["schedule_name"], two_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### iv.) 3 distance matches

In [ ]:
three_dist_matches = player_matches_df[player_matches_df["distance"] == 3]

In [ ]:
three_dist_matches

In [ ]:
name_list = ["Mackenzie McDonald", "Matthew Ebden", "Nicolas Mahut", "Bianca Andreescu", "Benoit Paire", "Ashleigh Barty"]
name_list += ["Jerzy Janowicz", "Natalia Vikhlyantseva", "Darian King", "Daniel Evans", "Jessica Moore"]
three_dist_matches = three_dist_matches[three_dist_matches["schedule_name"].isin(name_list)]

In [ ]:
update_dict = dict(zip(three_dist_matches["schedule_name"], three_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### v.) most frequent players

In [ ]:
already_found = list(true_matches.keys())

### TODO: Nekik még mindenféleképpen érdemes lenne utána keresni (valószínűleg kézzel)

In [ ]:
big_distance = player_matches_df[player_matches_df["distance"] > 3]
big_dist_frequent_player = big_distance[big_distance["schedule_count"] > 2]
big_dist_frequent_player = big_dist_frequent_player[~big_dist_frequent_player["schedule_name"].isin(already_found)]
len(big_dist_frequent_player)

In [ ]:
big_dist_frequent_player.head(50)
#big_dist_frequent_player.tail(38)

In [ ]:
name_list = ["Nick Kyrgios", "Victor Estrella Burgos", "Irina-Camelia Begu"]
name_list += ["Pierre-Hugues Herbert", "Zarina Diyas", "Pablo Carreno Busta", "Yung-Jan Chan"]
most_pop_players =  big_dist_frequent_player[ big_dist_frequent_player["schedule_name"].isin(name_list)]

In [ ]:
update_dict = dict(zip(most_pop_players["schedule_name"], most_pop_players["twitter_name"]))
true_matches.update(update_dict)

In [ ]:
len(true_matches)

true_matches

player_matches_df[player_matches_df["distance"]<3].head(10)

big_distance.head(10)

In [ ]:
sns.jointplot(x="schedule_count", y="distance", data=big_distance, kind="kde")

In [ ]:
sns.jointplot(x="schedule_count", y="distance", data=big_distance)

# Missing players

In [ ]:
missing_players_df = player_matches_df[~player_matches_df["schedule_name"].isin(true_matches)]

In [ ]:
missing_sch = dict(zip(missing_players_df["schedule_name"], missing_players_df["schedule_count"]))

In [ ]:
%%time
missing_edit_distances = np.zeros( (len(missing_sch), len(trg_dict_values)) )
missing_player_names = list(missing_players_df["schedule_name"])
for i in range(len(missing_sch)):
    reference = missing_player_names[i]
    missing_edit_distances[i, :] = [editdistance.eval("".join(reference.lower().split(" ")), "".join(comp.lower().split("_"))) for comp in trg_dict_values]

In [ ]:
missing_min_indices = missing_edit_distances.argmin(axis=1)
missing_player_matches = []
for i in range(len(missing_sch)):
    reference, min_match = missing_player_names[i], trg_dict_values[missing_min_indices[i]]
    missing_edit_dist = editdistance.eval( "".join(reference.lower().split(" ")), "".join(min_match.lower().split("_")) )
    missing_player_matches.append( (reference, missing_sch[reference], screen_dict_values[min_match], min_match, missing_edit_dist) )
missing_player_matches_df = pd.DataFrame(missing_player_matches, columns=["schedule_name", "schedule_count", "twitter_name", "screen_name", "distance"])

In [ ]:
missing_player_matches_df["distance"].value_counts().sort_index()

In [ ]:
missing_player_matches_df.sort_values("distance")

#### i.) Pure matches

In [ ]:
zero_dist_matches = missing_player_matches_df[missing_player_matches_df["distance"] == 0]

In [ ]:
update_dict = dict(zip(zero_dist_matches["schedule_name"], zero_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### ii.) 1 distance matches

In [ ]:
one_dist_matches = missing_player_matches_df[missing_player_matches_df["distance"]==1]
one_dist_matches

In [ ]:
update_dict = dict(zip(one_dist_matches["schedule_name"], one_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### iii.) 2 distance matches

In [ ]:
two_dist_matches = missing_player_matches_df[missing_player_matches_df["distance"] == 2]
two_dist_matches

In [ ]:
name_list = ["Sofia Kenin"] # , "Joao Sousa"
two_dist_matches = two_dist_matches[two_dist_matches["schedule_name"].isin(name_list)]

In [ ]:
update_dict = dict(zip(two_dist_matches["schedule_name"], two_dist_matches["twitter_name"]))
true_matches.update(update_dict)

# Joao Sousa and Souza ???????????????????????????????????

#### iv.) 3 distance matches

In [ ]:
three_dist_matches = missing_player_matches_df[missing_player_matches_df["distance"] == 3]

In [ ]:
three_dist_matches

In [ ]:
name_list = ["Nicolas Jarry"] # , "Joao Souza"]
three_dist_matches = three_dist_matches[three_dist_matches["schedule_name"].isin(name_list)]

In [ ]:
update_dict = dict(zip(three_dist_matches["schedule_name"], three_dist_matches["twitter_name"]))
true_matches.update(update_dict)

#### v.) >3 distance matches

In [ ]:
big_distance = missing_player_matches_df[missing_player_matches_df["distance"] > 3]
big_dist_frequent_player = big_distance[big_distance["schedule_count"] > 2]
big_dist_frequent_player = big_dist_frequent_player[~big_dist_frequent_player["schedule_name"].isin(already_found)]
len(big_dist_frequent_player)

In [ ]:
big_dist_frequent_player.head()
#big_dist_frequent_player.tail(38)

In [ ]:
len(true_matches)

In [ ]:
true_matches

In [ ]:
with open("../true_matches.json", "w") as f:
    json.dump(true_matches, f)

In [ ]:
true_matches_screen_names = list(mentions_df[mentions_df["trg_str"].isin(true_matches.values())]["trg_screen_str"].unique())

with open("../true_matches_screen_names.json", "w") as f:
    json.dump(true_matches_screen_names, f)

# Visualize missing players

In [ ]:
daily_players = {}
for index, row in schedule_df.iterrows():
    date, winner, loser = row["date"], row["playerName active"], row["playerName opponent"]
    court, match = row["courtName"], row["orderNumber"]
    match_id = "%s_%i" % (court, match)
    if not date in daily_players:
        daily_players[date] = {}
    else:
        daily_players[date][winner] = match_id
        daily_players[date][loser] = match_id

In [ ]:
schedule_df.head()

In [ ]:
daily_players.keys()

In [ ]:
daily_players['2017-06-11']

In [ ]:
daily_players_grouped = [(key, list(daily_players[key].keys())) for key in daily_players]

In [ ]:
daily_players_df = pd.DataFrame(daily_players_grouped, columns=["date", "players"])

In [ ]:
daily_players_df["count"] = daily_players_df.apply(lambda x: len(x["players"]), axis=1)

In [ ]:
def count_matches(player_list):
    num = 0
    for player in player_list:
        if player in true_matches:
            num += 1
    return num

In [ ]:
daily_players_df["match_count"] = daily_players_df.apply(lambda x: count_matches(x["players"]), axis=1)
daily_players_df["mismatch_count"] = daily_players_df.apply(lambda x: x["count"]-x["match_count"], axis=1)

In [ ]:
daily_players_df = daily_players_df.sort_values("date")

In [ ]:
len(true_matches)

In [ ]:
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(15, 10))

sns.set_color_codes("pastel")
sns.barplot(x="count", y="date", data=daily_players_df,
            label="Total count", color="b")

sns.set_color_codes("muted")
sns.barplot(x="match_count", y="date", data=daily_players_df,
            label="Match count", color="b")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="Date", xlabel="Number of tennis players")
sns.despine(left=True, bottom=True)

## Visualize missing players by rounds

In [ ]:
schedule_df.head()

In [ ]:
daily_match_players = {}
for index, row in schedule_df.iterrows():
    date, matchHeader, winner, loser = row["date"], row["matchHeader"], row["playerName active"], row["playerName opponent"]
    
    if not (date, matchHeader) in daily_match_players:
        daily_match_players[(date, matchHeader)] = []
    daily_match_players[(date, matchHeader)] += [winner, loser]
        

In [ ]:
daily_match_players_df = pd.DataFrame(pd.Series(daily_match_players), columns=["players"])
daily_match_players_df["count"] = daily_match_players_df["players"].apply(len)

In [ ]:
daily_match_players_df["match_count"] = daily_match_players_df["players"].apply(count_matches)
daily_match_players_df["mismatch_count"] = daily_match_players_df["count"] - daily_match_players_df["match_count"]

In [ ]:
daily_match_players_df["index"] = daily_match_players_df.index
daily_match_players_df["matchHeaders"] = daily_match_players_df["index"].apply(lambda t: t[1])
daily_match_players_df["date"] = daily_match_players_df["index"].apply(lambda t: t[0])

In [ ]:
def get_singles(value):
    return "Men's Singles" in value or "Women's Singles" in value

In [ ]:
def leave_b_g_wh(value):
    return "Boy" not in value and "Girl" not in value and "Wheelchair" not in value and "Legends Over 45" not in value

In [ ]:
daily_match_players_df = daily_match_players_df[daily_match_players_df["matchHeaders"].apply(leave_b_g_wh)]

daily_match_players_df = daily_match_players_df[daily_match_players_df["matchHeaders"].apply(get_singles)]

In [ ]:
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(15, 35))

sns.set_color_codes("pastel")
sns.barplot(x="count", y="index", data=daily_match_players_df,
            label="Total count", color="b")

sns.set_color_codes("muted")
sns.barplot(x="match_count", y="index", data=daily_match_players_df,
            label="Match count", color="b")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="Index", xlabel="Number of tennis players")
sns.despine(left=True, bottom=True)

# Search missing players

In [ ]:
new_matches = {"Tracy Austin-Holt": "Tracy Austin", "María José Martinez Sanchez": "Maria Jose Martinez", "Catherine Bellis": "CiCi Bellis",
               "Jan-Lennard Struff": "Jan-L. Struff", "Mirjana Lucic-Baroni": "Mirjana Lucic", "Ernesto Escobedo": "Netoesco",
               "Myrtille Georges": "GEORGES Myrtille", "Anastasia Pavlyuchenkova": "A. Pavlyuchenkova", "Tessah Andrianjafitrimo": "Tessah Andrian",
               "Gregoire Jacq": "JACQ Grégoire", "Nicholas Monroe": "Nick Monroe", "Risa Ozaki": "尾崎里紗/Risa Ozaki", "Miyu Kato": "Miyu Kato / 加藤 未唯",
               "Yuichi Sugita": "杉田祐一 / Yuichi SUGITA", "Danka Kovinic": "Kovinić Danka", "Misaki Doi": "Misaki Doi 土居美咲", 
               "Saisai Zheng": "Zheng Saisai", "Yen-Hsun Lu": "Rendy Lu", "Sara Sorribes Tormo": "Sara Sorribes", 
               "Maryna Zanevska": "Zanevska Maryna", "Montserrat Gonzalez": "Montse Gonzalez B.", "Harmony Tan": "Harmony", 
               "Leonardo Mayer": "Leo Mayer", "Corentin Moutet": "moutet corentin", "Lorenzo Giustino": "Lorenzo Giustino"}

In [ ]:
new_matches_wout_mentions = {"Anastasia Myskina": "Anastasia Myskina", "Arantxa Sanchez": "Arantxa Sanchez", "Katarina Srebotnik": "Katarina Srebotnik",
                             "Eri Hozumi": "Eri Hozumi × 穂積絵莉", "Chia-Jung Chuang": "Chia-Jung Chuang 莊佳容", "Brian Baker": "Brian Baker",
                             "Oscar Otte": "Oscar Otte", "Ksenia Lykina": "Ksenia Lykina", "Viktoriya Tomova": "Viktoriya Tomova",
                             "Maximilian Marterer": "Maximilian Marterer", "Marton Fucsovics": "Fucsovics Márton", "Ramkumar Ramanathan": "Ramkumar Ramanathan",
                             "Daniel Brands": "Daniel Brands", "Brydan Klein": "Brydan Klein", "Akira Santillan": "Akira Santillan", 
                             "Mitchell Krueger": "Mitchell Krueger", "Yuki Bhambri": "Yuki Bhambri", "Andrej Martin": "Andrej Martin", 
                             "Kimmer Coppejans": "Kimmer Coppejans", "John-Patrick Smith": "JP Smith", "Michael Mmoh": "Michael Mmoh", 
                             "Agustin Velotti": "Agustin Velotti", "Inigo Cervantes": "Iñigo Cervantes", "Hiroki Moriya": "Hiroki Moriya/守屋 宏紀", 
                             "Thomas Fabbiano": "Thomas Fabbiano", "Stefano Travaglia": "stefano travaglia", "Grega Zemlja": "Grega Zemlja", 
                             "Akiko Omae": "Akiko Omae", "Arina Rodionova": "Arina Rodionova", "Dalma Galfi": "Dalma Galfi", "Grace Min": "Grace Min", 
                             "Lesley Kerkhove": "Lesley Kerkhove", "Jamie Loeb": "Jamie Loeb", "Lizette Cabrera": "Lizette Cabrera",
                             "Alexa Glatch": "Alexa Glatch", "Alexandra Dulgheru": "Alexandra Dulgheru", "Conny Perrin": "Conny Perrin", 
                             "Misa Eguchi": "Misa Eguchi / 江口実沙", "Olga Govortsova": "Olga Govortsova", "Enrique Lopez-Perez": "Enrique lopez-Perez", 
                             "Ruben Bemelmans": "Ruben Bemelmans", "Marco Chiudinelli": "Marco Chiudinelli", "Jason Jung": "Jason Jung 莊吉生", 
                             "Henri Laaksonen": "Henri Laaksonen", "Daniel Gimeno-Traver": "Daniel Gimeno-Traver"}

In [ ]:
matches_not_found = ["Thomas Enqvist", "Julio Peralta", "Hao-Ching Chan", "Anastasija Sevastova", "Ying-Ying Duan", "Roman Jebavy", 
                     "Viktorija Golubic", "Chen Liang", "Mikhail Elgin", "Lukasz Kubot", "Yifan Xu", "Marcin Matkowski", "Oliver Marach", 
                     "Paolo Lorenzi", "Ekaterina Alexandrova", "Giulia Morlet", "Shuko Aoyama", "Raquel Atawo", "Denis Istomin", "Kveta Peschke", 
                     "Kateryna Bondarenko", "Johanna Larsson", "Natela Dzalamidze", "Mikhail Kukushkin", "Kenny De Schepper", "Jozef Kovalik",
                     "Katerina Siniakova", "Monica Niculescu", "Kristina Kucova", "Evgeny Donskoy", "Mikhail Youzhny", "Mischa Zverev",
                     "Kateryna Kozlova", "Gilles Muller", "Qiang Wang", "Evgeniya Rodina", "Matteo Donati", "Daniel Masur", "Peter Gojowczyk",
                     "Anna Blinkova", "Su Jeong Jang", "Barbara Haas", "Victoria Kamenskaya", "Alexandra Panova", "Tereza Martincova",
                     "Valentini Grammatikopoulou", "Elitsa Kostova", "Veronika Kudermetova", "Tereza Smitkova", "Marco Cecchinato",
                     "Jessika Ponchet", "Rebecca Sramkova", "Tamara Korpatsch", "Kai-Chen Chang", "Sabina Sharipova", "Viktoriya Tomova",
                     "Jasmine Paolini", "Anna Kalinskaya", "Jade Suvrijn", "Mirza Basic", "Laslo Djere", "Matija Pecotic", "Maximo Gonzalez",
                     "Vincent Millot", "Dmitry Popko", "Alessandro Giannessi", "Salvatore Caruso", "Ilya Ivashka", "Luca Vanni", "Vaclav Safranek",
                     "Duckhee Lee", "Roberto Carballes Baena", "Riccardo Bellotti", "Ruben Ramirez Hidalgo", "Blaz Rola", "Di Wu", "Federico Gaio",
                     "Mohamed Safwat", "Miljan Zekic", "Christian Garin", "Benjamin Becker", "Tatsuma Ito", "Yannik Reuter", "Daniel Munoz De La Nava",
                     "Sara Cakarevic", "Na-Lae Han", "Anhelina Kalinina", "Aryna Sabalenka", "Dalila Jakupovic", "Xinyun Han", "Isabella Shinikova",
                     "Cindy Burger", "Tamara Zidansek", "Riko Sawayanagi", "Sílvia Soler-Espinosa", "Anastasiya Komardina", "Kai-Lin Zhang", 
                     "Adam Pavlasek", "Aldin Setkic", "Adrian Menendez-Maceiras"]

In [ ]:
dont_know = {"Aliaksandra Sasnovich": "Sasnovich Sasha", "Stefano Napolitano": "", 
             "Joao Sousa": "joao pedro sousa", "Pedro Sousa": "joao pedro sousa", "Joao Souza": "joao pedro sousa",
             "Max Mirnyi": "Max Mirnyi", "Shuai Peng": "Pengshuai", "Patricia Maria Tig": "tig patricia maria", "Stephane Robert": "",
             "Hiroko Kuwata": "HIROKO KUWANA", "Lukas Rosol": "Lukas Rosol", "Lukas Lacko": "Lukáš Lacko", "Kristijan Mesaros": "Kristijan Mesaroš", 
             "Igor Sijsling": "igor sijsling", "Igor Sijsling": "IGOR SIJSLING"}

In [ ]:
players_list = sum(daily_match_players_df["players"], [])
players_list = list(filter(lambda p: p not in list(true_matches) + list(new_matches) + matches_not_found + list(new_matches_wout_mentions) + list(dont_know), players_list))
players_list

In [ ]:
mentions_df[mentions_df["trg_str"].apply(lambda n: "Traver".lower() in n.lower())].drop_duplicates("trg_str")

In [ ]:
def count_matches(player_list):
    num = 0
    for player in player_list:
        if player in true_matches:
            num += 1
    return num

In [ ]:
def count_matches(player_list):
    return len(list(filter(lambda v: v in list(true_matches)+list(new_matches), player_list)))

In [ ]:
daily_match_players_df["match_count"] = daily_match_players_df["players"].apply(count_matches)
daily_match_players_df["mismatch_count"] = daily_match_players_df["count"] - daily_match_players_df["match_count"]

In [ ]:
sns.set(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(15, 35))

sns.set_color_codes("pastel")
sns.barplot(x="count", y="index", data=daily_match_players_df,
            label="Total count", color="b")

sns.set_color_codes("muted")
sns.barplot(x="match_count", y="index", data=daily_match_players_df,
            label="Match count", color="b")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="Index", xlabel="Number of tennis players")
sns.despine(left=True, bottom=True)